# Kleio Walkthrough

## About

Author: Jared Neumann

This package is designed to take a PDF document with or without a text layer, or raw text, and return a complete, corrected version of that text. Text is extracted using common OCR tools, if necessary, and the text is then passed to an LLM. The LLM then makes corrections to each chunk. Additional functions can be called, such as:
- Layout analysis and annotation
- Revised collation (e.g., to eliminate headers and footers, etc.)
- Translation

## Import Statements

In [1]:
# we'll have to set up a duplicate logger in the notebook
import logging
import os

# Get the root logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create a StreamHandler for the notebook
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
stream_handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(stream_handler)

# set propagation to false to prevent double logging
logger.propagate = False

from kleio.ocr import *
from kleio.image_utils import *
from kleio.correction import *
from kleio.collation import *
from kleio.translation import *

import matplotlib.pyplot as plt

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## Getting Raw Text

There are a few allowable file types: PDF with text, PDF without text, images, and plain text. The type is automatically inferred, and the extracted text is returned. A few options for OCR are available.

In [2]:
example_filepath_0 = "../tests/test_input/test_0.pdf"
example_filepath_1 = "../tests/test_input/test_1.jpg"
example_filepath_2 = "../tests/test_input/test_2.pdf"
phil_mag_s1_v1_filepath = "../data/input/phil_mag_s1_v1.pdf"
output_dir = "../data/output"

IMAGE_CONFIG = {
    "grayscale": True,
    "resize": False,
    "threshold": True,
    "deskew": False,
    "dilate_and_erode": False,
    "blur": False
}

#text_0 = retrieve_text(example_filepath_0, image_kwargs=IMAGE_CONFIG)
#text_1 = retrieve_text(example_filepath_1, image_kwargs=IMAGE_CONFIG)
#text_2 = retrieve_text(example_filepath_2, image_kwargs=IMAGE_CONFIG)
text_3 = retrieve_text(phil_mag_s1_v1_filepath, image_kwargs=IMAGE_CONFIG)

2024-01-22 16:16:17,267 - kleio.ocr - INFO - Retrieving text from ../data/input/phil_mag_s1_v1.pdf


2024-01-22 16:16:17,267 - kleio.ocr - INFO - Retrieving text from ../data/input/phil_mag_s1_v1.pdf


2024-01-22 16:16:17,268 - kleio.ocr - INFO - File provided


2024-01-22 16:16:17,268 - kleio.ocr - INFO - File provided


2024-01-22 16:16:17,333 - kleio.ocr - INFO - Getting text from PDF file ../data/input/phil_mag_s1_v1.pdf


2024-01-22 16:16:17,333 - kleio.ocr - INFO - Getting text from PDF file ../data/input/phil_mag_s1_v1.pdf


In [3]:
# let's inspect one of the outputs
print(text_3)
print(len(text_3["pages"]))
for page in text_3["pages"]:
    print(page)

{'filename': 'phil_mag_s1_v1.pdf', 'extension': 'PDF', 'pages': ["mu \nTHE \nPHILOSOPHICAL MAGAZINE. \n' / V . \n• f . « f ( \nCOMPREHENDING \nTHE VARIOUS BRANCHES OF SCIENCE, \nTHE LIBERAL AND FINE ARTS, \nAGRICULTURE, MANUFACTURES, \nAnj) \nCOMMERCE. \n^ I \n-L—J. . ■ j  ;■■■■. \nBY ALEXANDER TILLOCH, \nMember of the London philosophical society. \n“ Nec aranearum fane textus ideo melior, quia ex fe fila gignunt. Nec nofter \n^ilior quia ex alienis libamus ut apes.” Just. Lips. Monit. Polit, lib. i. cap. \n- 9 \n.mmmmtmmm&KKBtKBmtmmrnm'. II. A1 \nVOL. I. \nLONDON: \nPrinted for the Proprietors : And fold by MefTrs. Richardson* \nCornhill; Cadlll and Davies, Strand; Debrett, Piccadilly ; \nMurray and Highley, No. 32, Fleet-ilreet; Symonds, \nPaternofter Row ; Bell, No. 148, Oxford-ftreet ; \nVernor and Hood, Poultry; Harding, No. 36, \nSt. James’s-ftreet; J. Remnant, Bigh-ftreet, \nSt, Giles’s; and W, Remnant, \nHamburgh. \n", ") \n• . \nl \nK \n» \n< \n\\ \nI \n' \nI \nt \ni\n", 'PRE

## Correcting the Raw Text

In [4]:

CORRECTION_KWARGS = {
    "filename": text_3["filename"],
    "extension": text_3["extension"],
    "filetype": "academic journal",
    "ocr_software": "pytesseract",
    "image_preprocessing_software": "opencv",
    "date": "1798",
    "language": "British English",
    "comments": "This is an article from the first volume of the Philosophical Magazine.",
}

In [6]:
correction = get_correction(
    text = text_3,
    api_key="not-needed",
    llm_provider="openai",
    model_name="mistralai/Mistral-7B-v0.1",
    base_url="http://localhost:1234/v1",
    temperature=0,
    more_info=CORRECTION_KWARGS,
    chunk_size=2048,
)

2024-01-22 16:17:40,902 - kleio.correction - INFO - Getting correction from LLM


2024-01-22 16:17:40,902 - kleio.correction - INFO - Getting correction from LLM


2024-01-22 16:17:40,904 - kleio.llm_utils - INFO - Creating OpenAI prompt for OCR correction task


2024-01-22 16:17:40,904 - kleio.llm_utils - INFO - Creating OpenAI prompt for OCR correction task


2024-01-22 16:17:40,905 - kleio.llm_utils - INFO - Creating OpenAI LLM


...

2024-01-22 16:19:07,732 - kleio.correction - INFO - Correcting chunks for page 3


2024-01-22 16:19:07,732 - kleio.correction - INFO - Correcting chunks for page 3
2024-01-22 16:19:29,429 - httpx - INFO - HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
  1%|          | 4/476 [00:46<1:46:45, 13.57s/it]

2024-01-22 16:19:29,434 - kleio.correction - INFO - Correcting chunks for page 4


2024-01-22 16:19:29,434 - kleio.correction - INFO - Correcting chunks for page 4
  1%|          | 4/476 [01:09<2:16:30, 17.35s/it]


KeyboardInterrupt: 

In [ ]:
for page in correction:
    print(page)

TypeError: 'NoneType' object is not iterable

## Collating the corrected pages

In [ ]:
# As you can see, there are some issues we might still want to resolve
# E.g., words that are split across lines,
# duplicate words on adjacent pages (old texts do this a lot),
# headers and footers that are not part of the main text, etc.

# So, we need to collate the pages
COLLATION_KWARGS = {
    "remove_headers_and_footers": True,
    "remove_page_numbers": True,
    "remove_excess_space": True,
    "remove_empty_lines": False,
    "remove_line_breaks": True,
    "remove_word_breaks": True,
    "add_section_tags": False,
    "keep_page_breaks": False,
}

collated_text = collate(
    pages = correction,
    api_key="not-needed",
    model_name="mistralai/Mistral-7B-v0.1",
    base_url="http://localhost:1234/v1",
    temperature=0,
    llm_provider="openai",
    chunk_size=2048,
    more_info=COLLATION_KWARGS,
)

2024-01-22 10:57:25,422 - kleio.llm_utils - INFO - Creating OpenAI prompt for OCR correction task


2024-01-22 10:57:25,422 - kleio.llm_utils - INFO - Creating OpenAI prompt for OCR correction task


2024-01-22 10:57:25,423 - kleio.llm_utils - INFO - Creating OpenAI LLM


2024-01-22 10:57:25,423 - kleio.llm_utils - INFO - Creating OpenAI LLM
100%|██████████| 1/1 [00:35<00:00, 35.04s/it]


In [ ]:
print(collated_text)

```python
"Received June 8, 1771.\nA Letter from Mr. Lane, Apothecary,\nin Aldergate-street,\nto the Honourable Henry Cavendish, R.S.\non\nthe Solubility of Iron in Simple Water,\n\nby\nDownloaded from https://royalsocietypublishing.org/\non 14 January 2024.\n\nsir,\nRead Nov. 23, 1771.\nThe various impregnations of mineral waters have always been very difficult to explain:\nand whoever has read the diverse and often contradictory reasonings upon the subject,\nmust clearly perceive that there is still room for discoveries in this part of natural history.*\nYou, Sir, by your accounts of fixed air,\nand of Rathbone-place water,\nrelated in the last volume of Philosophical Transactions,\nhave obliged the public with many additional lights on this branch of knowledge^ and,\nfrom your known accuracy and diligent pursuits in most philosophical inquiries,\nthe learned World has great reason to hope for many other new and useful improvements.*\nTo your judgment therefore, I submit the followin

## Translating the corrected, collated text

In [ ]:
TRANSLATION_KWARGS = {
    "target_language": "Spanish",
    "title": "On the Solubility of Iron in Simple, by the Intervention of fixed Air",
    "author": "Mr. Lane",
    "notes": "This is an article from the Philosophical Transactions of the Royal Society from 1770-1800",
}

translation = translate(
    text = collated_text,
    api_key="not-needed",
    llm_provider="openai",
    model_name="mistralai/Mistral-7B-v0.1",
    base_url="http://localhost:1234/v1",
    temperature=0.1,
    chunk_size=2048,
    more_info=TRANSLATION_KWARGS,
)

TypeError: translate() got an unexpected keyword argument 'base_url'

In [ ]:
print(translation)

Recibido el 8 de junio de 1770. Sr. Lane, boticario, calle Aldergate, al honorable Henry Cavendish, R.S., sobre la solubilidad del hierro en agua simple, por la intervención del aire fijo. Calle Aldergate, 5 de junio de 1769. Señor, leído el 23 de noviembre de 1769.

Las diversas impregnaciones de las aguas minerales siempre han sido muy difíciles de explicar; y quienquiera que haya leído los diversos y a menudo contradictorios razonamientos sobre este tema, debe percibir claramente que todavía hay espacio para descubrimientos en esta parte de la historia natural.

Usted, señor, con sus relatos sobre el aire fijo, y sobre el agua de Rathbone-place, relatados en el último volumen de las Transacciones Filosóficas, ha obligado al público con muchas luces adicionales sobre esta rama del conocimiento. Y, por su conocida precisión y diligentes investigaciones en la mayoría de las indagaciones filosóficas, el mundo erudito tiene grandes razones para esperar muchas otras mejoras nuevas y útile